In [12]:
from pathlib import Path
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import csv

In [13]:
def pull_from_csv(csv_file, row=3, header=1):
        lwp_ind = str(csv_file).lower().find('lwp')
        lwp = str(csv_file)[lwp_ind:lwp_ind+6]
        df = pd.read_csv(csv_file, header=[header], error_bad_lines=False)
        try:
            return [lwp,*df.iloc[row].name]
        except:
            return [lwp,*df.iloc[row]]
        
        
        
        
        
def met_ratio(df, up, down, method='Cramer', error_label=' Error (SD)'):
    if not isinstance(up,list):
        up = [up]
    if not isinstance(down,list):
        down = [down]    
   
    up_err_name = [f'{item}{error_label}' for item in up]
    down_err_name = [f'{item}{error_label}' for item in down]
    for item in np.concatenate([up_err_name, down_err_name]).flat:
        if item not in df.columns:
            raise NameError(f'Cannot find "{item}" in the DataFrame. Please check the given name is correct.')
    
    up_amp = df[up].astype(float).sum(axis=1)
    up_err = (df[up_err_name].astype(float)**2).sum(axis=1) ** 0.5
    down_amp = df[down].astype(float).sum(axis=1)
    down_err = (df[down_err_name].astype(float)**2).sum(axis=1) ** 0.5
    ratio = up_amp / down_amp
    ratio_err = ratio * ((up_err/up_amp)**2 + (down_err/down_amp)**2)**0.5 
    
    if len(up)>1:
        up_label = f'({"+".join(up)})'
    else:
        up_label = f'{"+".join(up)}'
    if len(down)>1:
        down_label = f'({"+".join(down)})'
    else:
        down_label = f'{"+".join(down)}'    
    col_name = f'{up_label}/{down_label}'   
    col_error_name = f'{col_name} Error (SD)'
    return pd.DataFrame({col_name: ratio, col_error_name: ratio_err})        

In [14]:
data_folder = Path('/Users/papo/Sync/MRdata/IoN_Piglet/MRS_RAY')
basis_folder = Path('/Users/papo/Sync/Projects/PAINT_MRS_CSI/3_0T_basis_threonine_no_MM')

sdat_files = [f for f in sorted(data_folder.rglob('*')) if ("act.sdat" in f.name.lower() and "csi" in f.name.lower())]
for file in tqdm(sdat_files):
    ref = Path(f'{str(file)[0:-8]}ref.SDAT')
    csv = Path(f'{str(file)[0:-5]}_both.csv')
    pdf = Path(f'{str(file)[0:-4]}pdf')
    if not csv.is_file():
        # command = f'tarquin --rows 3 --cols 2 --input {file} --output_pdf {pdf} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        command = f'tarquin --rows 3 --cols 2 --input {file} --output_csv {csv} --input_w {ref} --basis_csv {basis_folder}'
        print(f'{command}\n') 
        os.system(command)


100%|██████████| 22/22 [00:00<00:00, 4165.15it/s]


In [16]:
csv_files = [f for f in sorted(data_folder.rglob('*'), key=lambda x:x.parent.parent.stem[4:6]) if ("act_both.csv" in f.name.lower() and "csi" in f.name.lower())]



rows_to_process = {'BGT': 5, #(BGT = Basal Ganglia and Thalamic, ie deep GM)
                   'WM': 3,
                   }

for label,row_num in rows_to_process.items():
    data_list = []
    for file in csv_files:
        data_list.append([*pull_from_csv(file,row=row_num),*pull_from_csv(file,row=row_num+8),*pull_from_csv(file,header=16,row=row_num+1)])
    my_columns = list(pd.read_csv(file, header=[1], error_bad_lines=False))
    my_errors = [item+' Error (SD)' for item in my_columns]
    my_diagnostics = pull_from_csv(file,row=0, header=16)    
    df = pd.DataFrame(data_list, columns=['LWP', *my_columns, 'Error for LWP', *my_errors,*my_diagnostics ])
    df.to_excel(f'{label}.xlsx', index=False)
    
    #now calculate ratios
    df_L2N = met_ratio(df,['Lac', 'Threonine'], 'TNAA')
    df_N2C = met_ratio(df,'TNAA', 'TCho')
    df_C2C = met_ratio(df,'TCho', 'Cr')
    df_N2C = met_ratio(df,'TNAA', 'Cr')

    df_ratio = pd.concat((df_L2N, df_N2C, df_C2C, df_N2C, df), axis=1)
    df_ratio.set_index('LWP').to_excel(f'{label}_RATIOS.xlsx')



b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkipping line 20: expected 25 fields, saw 34\nSkipping line 21: expected 25 fields, saw 34\nSkipping line 22: expected 25 fields, saw 34\nSkipping line 23: expected 25 fields, saw 34\nSkipping line 24: expected 25 fields, saw 34\n'
b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkipping line 20: expected 25 fields, saw 34\nSkipping line 21: expected 25 fields, saw 34\nSkipping line 22: expected 25 fields, saw 34\nSkipping line 23: expected 25 fields, saw 34\nSkipping line 24: expected 25 fields, saw 34\n'
b'Skipping line 18: expected 25 fields, saw 34\nSkipping line 19: expected 25 fields, saw 34\nSkipping line 20: expected 25 fields, saw 34\nSkipping line 21: expected 25 fields, saw 34\nSkipping line 22: expected 25 fields, saw 34\nSkipping line 23: expected 25 fields, saw 34\nSkipping line 24: expected 25 fields, saw 34\n'
b'Skipping line 18: ex